In [7]:
%reload_ext autoreload

In [8]:
%cd "./drive/MyDrive/TC2008B"

# Adapted by Jorge Cruz from:
# TC2008B. Sistemas Multiagentes y Gráficas Computacionales
# Python server to interact with Unity
# Sergio. Julio 2021

[WinError 3] El sistema no puede encontrar la ruta especificada: './drive/MyDrive/TC2008B'
c:\Users\omarp\Downloads


In [9]:
%pip install pyngrok --quiet

from pyngrok import ngrok
from http.server import BaseHTTPRequestHandler, HTTPServer
import logging
import json
import os

import numpy as np

ngrok.install_ngrok()

# Terminate open tunnels if exist
ngrok.kill()

# Open an HTTPs tunnel on port 8585 for http://localhost:8585

port = os.environ.get("PORT", 8585)
server_address = ("", port)

things=[]

public_url = ngrok.connect(port="8585", proto="http", options={"bind_tls": True})
print("Tracking URL:", public_url)

INFO:pyngrok.process:Killing ngrok process: 22352
INFO:pyngrok.ngrok:Opening tunnel named: http-80-0150c694-b4c6-4352-9886-b1140ed2a1f5
INFO:pyngrok.process.ngrok:t=2021-09-09T22:48:28-0500 lvl=info msg="no configuration paths supplied"
INFO:pyngrok.process.ngrok:t=2021-09-09T22:48:28-0500 lvl=info msg="using configuration at default config path" path=C:\\Users\\omarp/.ngrok2/ngrok.yml
INFO:pyngrok.process.ngrok:t=2021-09-09T22:48:28-0500 lvl=info msg="open config file" path=C:\\Users\\omarp\\.ngrok2\\ngrok.yml err=nil
INFO:pyngrok.process.ngrok:t=2021-09-09T22:48:28-0500 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4041


Note: you may need to restart the kernel to use updated packages.


INFO:pyngrok.process.ngrok:t=2021-09-09T22:48:28-0500 lvl=info msg="tunnel session started" obj=tunnels.session
INFO:pyngrok.process.ngrok:t=2021-09-09T22:48:28-0500 lvl=info msg="client session established" obj=csess id=451439553ad4
INFO:pyngrok.process.ngrok:t=2021-09-09T22:48:28-0500 lvl=info msg=start pg=/api/tunnels id=f0156ce1b4465677
INFO:pyngrok.process.ngrok:t=2021-09-09T22:48:28-0500 lvl=info msg=end pg=/api/tunnels id=f0156ce1b4465677 status=200 dur=519.7Âµs
INFO:pyngrok.process.ngrok:t=2021-09-09T22:48:28-0500 lvl=info msg=start pg=/api/tunnels id=30b18ae92fbf671f
INFO:pyngrok.process.ngrok:t=2021-09-09T22:48:28-0500 lvl=info msg=end pg=/api/tunnels id=30b18ae92fbf671f status=200 dur=828.2Âµs
INFO:pyngrok.process.ngrok:t=2021-09-09T22:48:28-0500 lvl=info msg=start pg=/api/tunnels id=52e3e72f50e6732a
INFO:pyngrok.process.ngrok:t=2021-09-09T22:48:28-0500 lvl=info msg="started tunnel" obj=tunnels name="http-80-0150c694-b4c6-4352-9886-b1140ed2a1f5 (http)" addr=http://localhost:

Tracking URL: NgrokTunnel: "http://bdd3-2806-108e-13-5250-483e-334b-9875-d63.ngrok.io" -> "http://localhost:80"


INFO:pyngrok.process.ngrok:t=2021-09-09T22:48:28-0500 lvl=info msg=end pg="/api/tunnels/http-80-0150c694-b4c6-4352-9886-b1140ed2a1f5 (http)" id=d0cdfe1d9f96b571 status=200 dur=0s


In [10]:

# Commented out IPython magic to ensure Python compatibility.
from mesa import Agent, Model 
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector

import numpy as np
import pandas as pd
import random

import time
import datetime

"""## Crear el modelo"""

randomDireccion=0
switchSemaforo=False
auxSwitch=False
movimientosT=0

#Función que te devuelve la habitación (calle e intersección), ayuda a graficar el resultado
def obtener_habitacion(modelo):
    habitacion = np.zeros((modelo.grid.width, modelo.grid.height))
    for celda in modelo.grid.coord_iter():
        contenido_celda, x, y = celda
        for contenido in contenido_celda:
            if isinstance(contenido, Coche):
                habitacion[x][y] = 5
            else:
                if isinstance(contenido,Semaforo):
                    habitacion[x][y]=contenido.estadoS
                else:
                    habitacion[x][y] = contenido.estado
    return habitacion
    

#Agente Coche---------------------------------------------------------------------------------------------------------------------------------------------------------------
class Coche(Agent):
    #Constructor del agente Coche, el cual inicializa sus valores
    def __init__(self, id_unico, modelo):
        super().__init__(id_unico, modelo)
        self.id_unico=id_unico
        self.nueva_posicion = None
        self.movimientos = 0
        self.sentidoCOCHE=None
  
    #Función que define el estado que tendrá el coche para la siguiente iteración sin asignarlo rodavía
    #Esta función puede funcionar como un "pre-cambio" donde solo se asignan valores
    def step(self):
        global randomDireccion
        global auxSwitch

        vecinos = self.model.grid.get_neighbors(
            self.pos,
            moore=False,
            include_center=True)
        for vecino in vecinos:
            if isinstance(vecino,Coche) and self.sentidoCOCHE==vecino.sentidoCOCHE:
                self.nueva_posicion=self.pos
        for vecino in vecinos:
            #En caso de que haya un semáforo entre los vecinos
            if isinstance(vecino,Semaforo):
                #Si el semáforo esta en rojo o amarillo el auto se detiene
                if (vecino.estadoS==vecino.ROJO or vecino.estadoS==vecino.AMARILLO) and vecino.sentidoSEMAFORO==self.sentidoCOCHE:
                    self.nueva_posicion=self.pos
                    break
                #Si el semáforo se encuentra en verde el auto puede avanzar, también
                #escoge la dirección o camino que tomará
                if vecino.estadoS==vecino.VERDE and vecino.sentidoSEMAFORO==self.sentidoCOCHE:
                    randomDireccion=random.randint(1,3)
                    vecinos2 = self.model.grid.get_neighbors(
                        self.pos,
                        moore=False,
                        include_center=True)
                    for vecino2 in vecinos2:
                        if isinstance(vecino, Piso) and vecino.pos==self.pos:
                            vecino.siguiente_estado = vecino.estado
                            if vecino.siguiente_estado==vecino.CALLE:
                                #Se moverá del semáforo dependiendo que sentido tiene la calle
                                if vecino.SENTIDO_CALLE==vecino.SENTIDO_Este:
                                    self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(0,1)))
                                    self.sentidoCOCHE="E"
                                    break

                                if vecino.SENTIDO_CALLE==vecino.SENTIDO_Oeste:
                                    self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(0,-1)))
                                    self.sentidoCOCHE="O"
                                    break

                                if vecino.SENTIDO_CALLE==vecino.SENTIDO_Norte:
                                    self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(1,0)))
                                    self.sentidoCOCHE="N"
                                    break

                                if vecino.SENTIDO_CALLE==vecino.SENTIDO_Sur:
                                    self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(-1,0)))
                                    self.sentidoCOCHE="S"
                                    break
            else:
                #Si no exsiten semáforos cerca este se tendrá en cuenta como calle, donde el auto
                #puede circular sin restricciones
                if isinstance(vecino, Piso) and vecino.pos==self.pos:
                    vecino.siguiente_estado = vecino.estado
                    if vecino.siguiente_estado==vecino.CALLE:
                        #Se moverá en base al sentido de la calle
                        if vecino.SENTIDO_CALLE==vecino.SENTIDO_Este:
                            self.sentidoCOCHE="E"
                            self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(0,1)))
                                
                        if vecino.SENTIDO_CALLE==vecino.SENTIDO_Oeste:
                            self.sentidoCOCHE="O"
                            self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(0,-1)))
                            
                        if vecino.SENTIDO_CALLE==vecino.SENTIDO_Norte:
                            self.sentidoCOCHE="N"
                            self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(1,0)))
                             
                        if vecino.SENTIDO_CALLE==vecino.SENTIDO_Sur:
                            self.sentidoCOCHE="S"
                            self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(-1,0)))

                        if vecino.SENTIDO_CALLE==vecino.SENTIDO_TODOS:
                            #Vuelta continua a la derecha
                            if randomDireccion==1:
                                if self.sentidoCOCHE == "N":
                                    self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(0,-1)))
                                    self.sentidoCOCHE="O"
                                    vecinos3 = self.model.grid.get_neighbors(
                                        self.pos,moore=False,
                                        include_center=True)
                                    for vecino3 in vecinos3:
                                        if isinstance(vecino3,Piso) and vecino3.SENTIDO_CALLE==vecino3.SENTIDO_Oeste:
                                            auxSwitch=True
                                    break
                                if self.sentidoCOCHE == "O":
                                    self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(-1,0)))
                                    self.sentidoCOCHE="S"
                                    vecinos3 = self.model.grid.get_neighbors(
                                        self.pos,moore=False,
                                        include_center=True)
                                    for vecino3 in vecinos3:
                                        if isinstance(vecino3,Piso) and vecino3.SENTIDO_CALLE==vecino3.SENTIDO_Sur:
                                            auxSwitch=True
                                    break
                                if self.sentidoCOCHE == "S":
                                    self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(0,1)))
                                    self.sentidoCOCHE="E"
                                    vecinos3 = self.model.grid.get_neighbors(
                                        self.pos,moore=False,
                                        include_center=True)
                                    for vecino3 in vecinos3:
                                        if isinstance(vecino3,Piso) and vecino3.SENTIDO_CALLE==vecino3.SENTIDO_Este:
                                            auxSwitch=True
                                    break
                                if self.sentidoCOCHE == "E":
                                    self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(1,0)))
                                    self.sentidoCOCHE="N"
                                    vecinos3 = self.model.grid.get_neighbors(
                                        self.pos,moore=False,
                                        include_center=True)
                                    for vecino3 in vecinos3:
                                        if isinstance(vecino3,Piso) and vecino3.SENTIDO_CALLE==vecino3.SENTIDO_Norte:
                                            auxSwitch=True
                                    break
                            #El auto irá en la misma dirección a la que se encontraba
                            if randomDireccion==2:
                                if self.sentidoCOCHE == "E":
                                    self.sentidoCOCHE="E"
                                    self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(0,1)))
                                    vecinos3 = self.model.grid.get_neighbors(
                                        self.pos,moore=False,
                                        include_center=True)
                                    for vecino3 in vecinos3:
                                        if isinstance(vecino3,Piso) and vecino3.SENTIDO_CALLE==vecino3.SENTIDO_Este:
                                            auxSwitch=True

                                if self.sentidoCOCHE == "O":
                                    self.sentidoCOCHE="O"
                                    self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(0,-1)))
                                    vecinos3 = self.model.grid.get_neighbors(
                                        self.pos,moore=False,
                                        include_center=True)
                                    for vecino3 in vecinos3:
                                        if isinstance(vecino3,Piso) and vecino3.SENTIDO_CALLE==vecino3.SENTIDO_Oeste:
                                            auxSwitch=True

                                if self.sentidoCOCHE == "N":
                                    self.sentidoCOCHE="N"
                                    self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(1,0)))
                                    vecinos3 = self.model.grid.get_neighbors(
                                        self.pos,moore=False,
                                        include_center=True)
                                    for vecino3 in vecinos3:
                                        if isinstance(vecino3,Piso) and vecino3.SENTIDO_CALLE==vecino3.SENTIDO_Norte:
                                            auxSwitch=True

                                if self.sentidoCOCHE == "S":
                                    self.sentidoCOCHE="S"
                                    self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(-1,0)))
                                    vecinos3 = self.model.grid.get_neighbors(
                                        self.pos,moore=False,
                                        include_center=True)
                                    for vecino3 in vecinos3:
                                        if isinstance(vecino3,Piso) and vecino3.SENTIDO_CALLE==vecino3.SENTIDO_Sur:
                                            auxSwitch=True

                            if randomDireccion==3:
                                if self.sentidoCOCHE == "N":
                                    #self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(1,2)))
                                    self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(1,0)))
                                    # self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(0,1)))
                                    # self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(0,1)))
                                    self.sentidoCOCHE="E"
                                    randomDireccion = 2
                                    vecinos3 = self.model.grid.get_neighbors(
                                        self.pos,moore=False,
                                        include_center=True)
                                    for vecino3 in vecinos3:
                                        if isinstance(vecino3,Piso) and vecino3.SENTIDO_CALLE==vecino3.SENTIDO_Oeste:
                                            auxSwitch=True
                                    break
                                if self.sentidoCOCHE == "O":
                                    # self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(2,-1)))
                                    self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(0,-1)))
                                    # self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(1,0)))
                                    # self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(1,0)))
                                    self.sentidoCOCHE="N"
                                    randomDireccion = 2
                                    vecinos3 = self.model.grid.get_neighbors(
                                        self.pos,moore=False,
                                        include_center=True)
                                    for vecino3 in vecinos3:
                                        if isinstance(vecino3,Piso) and vecino3.SENTIDO_CALLE==vecino3.SENTIDO_Norte:
                                            auxSwitch=True
                                    break
                                if self.sentidoCOCHE == "S":
                                    # self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(-1,-2)))
                                    self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(-1,0)))
                                    # self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(0,-1)))
                                    # self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(0,-1)))
                                    self.sentidoCOCHE="O"
                                    randomDireccion = 2
                                    vecinos3 = self.model.grid.get_neighbors(
                                        self.pos,moore=False,
                                        include_center=True)
                                    for vecino3 in vecinos3:
                                        if isinstance(vecino3,Piso) and vecino3.SENTIDO_CALLE==vecino3.SENTIDO_Oeste:
                                            auxSwitch=True
                                    break
                                if self.sentidoCOCHE == "E":
                                    # self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(-2,1)))
                                    self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(0,1)))
                                    # self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(-1,0)))
                                    # self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(-1,0)))
                                    self.sentidoCOCHE="S"
                                    randomDireccion = 2
                                    vecinos3 = self.model.grid.get_neighbors(
                                        self.pos,moore=False,
                                        include_center=True)
                                    for vecino3 in vecinos3:
                                        if isinstance(vecino3,Piso) and vecino3.SENTIDO_CALLE==vecino3.SENTIDO_Sur:
                                            auxSwitch=True
                                    break

                          
        
    def advance(self):
        global auxSwitch
        global switchSemaforo
        global movimientosT
        global things
        
        #Se actualiza la pocisión y la cantidad de movimientos
        if self.pos != self.nueva_posicion:
            self.movimientos = self.movimientos + 1
            movimientosT=movimientosT+1
        
        #Cambia el estado del semáforo en caso de haber un auto ahí
        if auxSwitch:
            switchSemaforo=False
        
          
        # Movemos el auto a su nueva posicion
        self.model.grid.move_agent(self, self.nueva_posicion)

        #CREAR DICCIONARIO Y DEVOLVERLO AQUÍ
        diccionarie={'tag':'coche','posx':self.nueva_posicion[0],'posz':self.nueva_posicion[1],'lights':0,'reference':self.id_unico}
        things.append(diccionarie)
        
    

#Agente Piso -----------------------------------------------------------------------------------------------------------------------------------------------
class Piso(Agent):
    
    CALLE = 1
    NO_CALLE = 0

    SENTIDO_Sur="S"
    SENTIDO_Norte="N"
    SENTIDO_Este="E"
    SENTIDO_Oeste="O"
    SENTIDO_TODOS="T"
    
    #Constructor del agente Piso, el cual inicializa sus valores
    def __init__(self, pos, modelo, estado=NO_CALLE):
        super().__init__(pos, modelo)
        self.x, self.y = pos
        self.estado = estado
        self.siguiente_estado = None
        self.SENTIDO_CALLE=None

#AGREGAR AGENTE SEMAFORO (TENDRA 3 estados (luces) ROJO=2 AMARILLO=3 VERDE=4)-----------------------------------------------------------------------------------------
class Semaforo(Agent):
    ROJO=2
    AMARILLO=3
    VERDE=4

    #Constructor del agente Semaforo, el cual inicializa sus valores
    def __init__(self,id_unico,modelo,estadoS=ROJO):
        super().__init__(id_unico,modelo)
        self.id_unico=id_unico
        self.estadoS=estadoS
        self.siguienteEstadoS=None
        self.sentidoSEMAFORO=None

    #Función que definirá el color que tomará el semáforo, solo se pre-asignarán valores
    def step(self):
        global switchSemaforo

        vecinos = self.model.grid.get_neighbors(
              self.pos,
              moore=True,
              include_center=False)
        for vecinoSem in vecinos:
            if isinstance(vecinoSem,Coche) and self.sentidoSEMAFORO==vecinoSem.sentidoCOCHE and not switchSemaforo:
                auxSwitch=False
                switchSemaforo=True
                self.siguienteEstadoS=self.VERDE
                break
            else:
                vecinosR = self.model.grid.get_neighbors(
                  self.pos,
                  moore=True,
                  include_center=False)
                for vecinoR in vecinosR:
                    if not isinstance(vecinoR,Coche) and not switchSemaforo:
                        self.siguienteEstadoS=self.AMARILLO
                        
                    else: 
                        self.siguienteEstadoS=self.ROJO

    #Se generan los cambios asignados en la función step
    def advance(self):
        global things

        diccionarie={'tag':'semaforo','posx':0,'posz':0,'lights':self.siguienteEstadoS,'reference':self.id_unico}
        things.append(diccionarie)

        self.estadoS=self.siguienteEstadoS


#Modelo-------------------------------------------------------------------------------------------------------------------------------------------------------------
class Habitacion(Model):
    #Constructor del modelo, el cual inicializa sus valores
    def __init__(self, m=10, n=10, num_agentes=4):
        self.num_agentes = num_agentes
        self.grid = MultiGrid(m, n, True)
        self.schedule = SimultaneousActivation(self)
               
        #Crear la calle/interseccion del modelo
        auxM2=(m/2)-1
        auxM1=(m/2)
        auxN2=(n/2)-1
        auxN1=(n/2)
        for celdas1 in range(m):
            for celdas2 in range(n):
                celda_posicion=(celdas1,celdas2)
                if celdas1 == auxM2 and celdas2 == auxN2:
                    piso= Piso(celda_posicion, self)
                    piso.estado=piso.CALLE
                    piso.SENTIDO_CALLE=piso.SENTIDO_TODOS
                    self.grid.place_agent(piso, celda_posicion)
                    self.schedule.add(piso)
                    continue

                if celdas1 == auxM1 and celdas2 == auxN1:
                    piso= Piso(celda_posicion, self)
                    piso.estado=piso.CALLE
                    piso.SENTIDO_CALLE=piso.SENTIDO_TODOS
                    self.grid.place_agent(piso, celda_posicion)
                    self.schedule.add(piso)
                    continue

                if celdas1 == auxM2 and celdas2 == auxN1:
                    piso= Piso(celda_posicion, self)
                    piso.estado=piso.CALLE
                    piso.SENTIDO_CALLE=piso.SENTIDO_TODOS
                    self.grid.place_agent(piso, celda_posicion)
                    self.schedule.add(piso)
                    continue

                if celdas1 == auxM1 and celdas2 == auxN2:
                    piso= Piso(celda_posicion, self)
                    piso.estado=piso.CALLE
                    piso.SENTIDO_CALLE=piso.SENTIDO_TODOS
                    self.grid.place_agent(piso, celda_posicion)
                    self.schedule.add(piso)
                    continue

                if celdas1 == auxM2:
                    piso= Piso(celda_posicion, self)
                    piso.estado=piso.CALLE
                    piso.SENTIDO_CALLE=piso.SENTIDO_Oeste
                    self.grid.place_agent(piso, celda_posicion)
                    self.schedule.add(piso)
                    continue

                if celdas1 == auxM1:
                    piso= Piso(celda_posicion, self)
                    piso.estado=piso.CALLE
                    piso.SENTIDO_CALLE=piso.SENTIDO_Este
                    self.grid.place_agent(piso, celda_posicion)
                    self.schedule.add(piso)
                    continue

                if celdas2 == auxN2:
                    piso= Piso(celda_posicion, self)
                    piso.estado=piso.CALLE
                    piso.SENTIDO_CALLE=piso.SENTIDO_Norte
                    self.grid.place_agent(piso, celda_posicion)
                    self.schedule.add(piso)
                    continue

                if celdas2 == auxN1:
                    piso= Piso(celda_posicion, self)
                    piso.estado=piso.CALLE
                    piso.SENTIDO_CALLE=piso.SENTIDO_Sur
                    self.grid.place_agent(piso, celda_posicion)
                    self.schedule.add(piso)
                    continue
        
        # Posicionar celdas donde no puede ir el coche (casas, edificios, etc)
        lista_celdas_vacias = list(self.grid.empties)
        for celdas in lista_celdas_vacias:
            if celdas==(int(auxM2-1),int(auxN2-1)) or celdas==(int(auxM2-1),int(auxN1+1)) or celdas==(int(auxM1+1),int(auxN2-1)) or celdas==(int(auxM1+1),int(auxN1+1)):
                continue
            else:
                piso = Piso(celdas, self)
                self.grid.place_agent(piso, celdas)
                self.schedule.add(piso)
        
        # Posicionar coches
        i=0
        for cocheP1 in range(m):
            for cocheP2 in range(n):
                if i<self.num_agentes:
                    if cocheP1==auxM1 and cocheP2==0:
                        i=i+1
                        coche = Coche(i, self)
                        self.grid.place_agent(coche, (cocheP1,cocheP2))
                        self.schedule.add(coche)
                        continue
                    if cocheP1==0 and cocheP2==auxN2:
                        i=i+1
                        coche = Coche(i, self)
                        self.grid.place_agent(coche, (cocheP1,cocheP2))
                        self.schedule.add(coche)
                        continue
                    if cocheP1==m-1 and cocheP2==auxN1:
                        i=i+1
                        coche = Coche(i, self)
                        self.grid.place_agent(coche, (cocheP1,cocheP2))
                        self.schedule.add(coche)
                        continue
                    if cocheP1==auxM2 and cocheP2==n-1:
                        i=i+1
                        coche = Coche(i, self)
                        self.grid.place_agent(coche, (cocheP1,cocheP2))
                        self.schedule.add(coche)
                        continue
                else:
                    break

        #Pocisionar semáforos
        i=0
        posiciones_semaforo=[(int(auxM2-1),int(auxN2-1)),
                     (int(auxM2-1),int(auxN1+1)),
                     (int(auxM1+1),int(auxN2-1)),
                     (int(auxM1+1),int(auxN1+1))]

        for semas in posiciones_semaforo:
            i=i+1
            semaforo = Semaforo(i+4, self)
            if i==1:
                semaforo.sentidoSEMAFORO="N"
            if i==2:
                semaforo.sentidoSEMAFORO="O"
            if i==3:
                semaforo.sentidoSEMAFORO="E"
            if i==4:
                semaforo.sentidoSEMAFORO="S"
            self.grid.place_agent(semaforo, semas)
            self.schedule.add(semaforo)


        self.colectordatos = DataCollector(
            model_reporters={'Habitacion': obtener_habitacion},
            agent_reporters={'Movimientos': lambda a: getattr(a, 'movimientos', None)}
        )
    
    def step(self):
        self.colectordatos.collect(self)
        self.schedule.step()

In [11]:
ngrok.kill()

#Inicializar el modelo
modelo=Habitacion(20,20,4)

def updateThings():
    global modelo
    global things

    #Hacer un clear a la lista de cosas cada que se llame a la función para solo
    #guardar las cosas que se hicieron en el step
    things.clear()

    #Actualiza pocisiones y todo eso, es como el step() que tenemos con Edgar
    #Actualiza todas las pocisiones de los objetos, por eso es for boid in flock
    #Le da la vuelta a todos los objetos creados en el arreglo flock
    #Probablemente tengamos que hacer algo parecido acá pero con un simple step() y advance()
    #Guardar las pocisiones de los vehículos en la función advance() y retornarlas acá
    #Dar estas dos instrucciones aquí para evitar desmadre y guardar todo
    #borrar comentarios para hacer el cambio cuando se este seguro de esto
    #Se hace el step del modelo, faltaría crear los diccionarios
    modelo.step()
    return things

def infoToJSON(ps):
    infoDict=[]
    for tModel in things:
        thing={
            "x" : tModel['posx'],
            "z" : tModel['posz'],
            "tag" : tModel['tag'],
            "lights" : tModel['lights'],
            "reference" : tModel['reference']
        }
        infoDict.append(tModel)
    return json.dumps(infoDict)


class Server(BaseHTTPRequestHandler):
    
    def _set_response(self):
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()
        
    def do_GET(self):
        logging.info("GET request,\nPath: %s\nHeaders:\n%s\n", str(self.path), str(self.headers))
        self._set_response()
        self.wfile.write("GET request for {}".format(self.path).encode('utf-8'))

    def do_POST(self):
        content_length = int(self.headers['Content-Length'])
        #post_data = self.rfile.read(content_length)
        post_data = json.loads(self.rfile.read(content_length))
        #logging.info("POST request,\nPath: %s\nHeaders:\n%s\n\nBody:\n%s\n",
                     #str(self.path), str(self.headers), post_data.decode('utf-8'))
        logging.info("POST request,\nPath: %s\nHeaders:\n%s\n\nBody:\n%s\n",
                     str(self.path), str(self.headers), json.dumps(post_data))
        
        '''
        x = post_data['x'] * 2
        y = post_data['y'] * 2
        z = post_data['z'] * 2
        
        position = {
            "x" : x,
            "y" : y,
            "z" : z
        }

        self._set_response()
        #self.wfile.write("POST request for {}".format(self.path).encode('utf-8'))
        self.wfile.write(str(position).encode('utf-8'))
        '''
        
        info = updateThings()
        #print(positions)
        self._set_response()
        print(infoToJSON(info))
        resp = "{\"data\":" +  infoToJSON(info) + "}"
        #print(resp)
        self.wfile.write(resp.encode('utf-8'))

def run(server_class=HTTPServer, handler_class=Server, port=8585):
    logging.basicConfig(level=logging.INFO)
    server_address = ('', port)
    httpd = server_class(server_address, handler_class)

    public_url = ngrok.connect(port).public_url
    logging.info("ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

    logging.info("Starting httpd...\n") # HTTPD is HTTP Daemon!
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:   # CTRL+C stops the server
        pass
    httpd.server_close()
    logging.info("Stopping httpd...\n")

#if __name__ == '__main__':
#    from sys import argv
#    
#    if len(argv) == 2:
#        run(port=int(argv[1]))
#    else:
#        run()


INFO:pyngrok.process:Killing ngrok process: 19872


In [12]:
run(HTTPServer, Server)

INFO:pyngrok.ngrok:Opening tunnel named: http-8585-76baa3d6-9088-468a-acd2-995247d4a7b1
INFO:pyngrok.process.ngrok:t=2021-09-09T22:48:29-0500 lvl=info msg="no configuration paths supplied"
INFO:pyngrok.process.ngrok:t=2021-09-09T22:48:29-0500 lvl=info msg="using configuration at default config path" path=C:\\Users\\omarp/.ngrok2/ngrok.yml
INFO:pyngrok.process.ngrok:t=2021-09-09T22:48:29-0500 lvl=info msg="open config file" path=C:\\Users\\omarp\\.ngrok2\\ngrok.yml err=nil
INFO:pyngrok.process.ngrok:t=2021-09-09T22:48:29-0500 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4041
INFO:pyngrok.process.ngrok:t=2021-09-09T22:48:29-0500 lvl=info msg="tunnel session started" obj=tunnels.session
INFO:pyngrok.process.ngrok:t=2021-09-09T22:48:29-0500 lvl=info msg="client session established" obj=csess id=6137ade2eb96
INFO:pyngrok.process.ngrok:t=2021-09-09T22:48:29-0500 lvl=info msg=start pg=/api/tunnels id=6219f128669ff2a4
INFO:pyngrok.process.ngrok:t=2021-09-09T22:48:29-0500 lvl=info